# Developing/Testing QA Frame Class
    v1.1 -- Flats, skysub
    v1.2 -- Bias

In [19]:
# imports
from importlib import reload
import os

from desispec.qa import qa_frame
from desispec.io import read_qa_frame
from desispec.io import qa as desio_qa

## Instantiate

In [7]:
reload(qa_frame)
# Dummy dict
fdict = {'20200304': {53027: {'flavor': 'arc', 'z3': {}}}}
#
qaframe = qa_frame.QA_Frame(fdict)
print(qaframe)

QA_Frame: night=20200304, expid=53027, camera=z3, flavor=arc


## Init SkySub

In [10]:
fdict = {'20200304': {53027: {'flavor': 'science', 'z3': {}}}}
#
qaframe = qa_frame.QA_Frame(fdict)
qaframe.init_skysub()
print(qaframe.qa_data)

{'SKYSUB': {'PARAMS': {'PCHI_RESID': 0.05, 'PER_RESID': 95.0, 'BIN_SZ': 0.1, 'SKYRESID_WARN_RANGE': [-5.0, 5.0], 'SKYRESID_ALARM_RANGE': [-10.0, 10.0], 'B_CONT': [[4000.0, 4500.0], [5250.0, 5550.0]], 'R_CONT': [[5950.0, 6200.0], [6990.0, 7230.0]], 'Z_CONT': [[8120.0, 8270.0], [9110.0, 9280.0]], 'SKYCONT_WARN_RANGE': [100.0, 400.0], 'SKYCONT_ALARM_RANGE': [50.0, 600.0]}}}


## I/O

In [14]:
desio_qa.write_qa_frame('tmp.yaml', qaframe)

'tmp.yaml'

In [15]:
qaframe = desio_qa.read_qa_frame('tmp.yaml')

In [16]:
qaframe

QA_Frame: night=20200304, expid=53027, camera=z3, flavor=science

# Bias QA

## Exposure

In [18]:
dpath = '/data/DESI/DESI_SCRATCH/minisv2/data/20200304'

In [20]:
dfile = os.path.join(dpath, '00053026', 'desi-00053025.fits.fz')

----

## Test FiberFlat QA

### On JXP laptop first

    desi_compute_fiberflat.py \
        --infile /Users/xavier/DESI/TST/dogwood/exposures/20150211/00000001/frame-b0-00000001.fits \
        --outfile /Users/xavier/DESI/TST/dogwood/calib2d/20150211/jxp-fflat-b0-00000001.fits \
        --fibermap /Users/xavier/DESI/TST/data/20150211/fibermap-00000002.fits \
        --qafile /Users/xavier/DESI/TST/dogwood/exposures/20150211/00000001/qa-b0-00000001.yaml \
        --qafig /Users/xavier/DESI/TST/dogwood/exposures/20150211/00000001/qa-fflat-b0-00000001.pdf

### Now Edison (see below for setup)

## Test SkySub QA

### On JXP laptop first

    desi_compute_sky.py \
        --infile /Users/xavier/DESI/TST/dogwood/exposures/20150211/00000002/frame-b0-00000002.fits \
        --fibermap /Users/xavier/DESI/TST/data/20150211/fibermap-00000002.fits \
        --fiberflat /Users/xavier/DESI/TST/dogwood/calib2d/20150211/fiberflat-b0-00000001.fits \
        --outfile /Users/xavier/DESI/TST/dogwood/exposures/20150211/00000002/sky2-b0-00000002.fits \
        --qafile /Users/xavier/DESI/TST/dogwood/exposures/20150211/00000002/qa-b0-00000002.yaml \
        --qafig /Users/xavier/DESI/TST/dogwood/exposures/20150211/00000002/qa-b0-00000002.pdf \
        > tmp.log

In [11]:
# Read and print
qaframe = desio_qa.read_qa_frame('/Users/xavier/DESI/TST/dogwood/exposures/20150211/00000002/qa-b0-00000002.yaml')
qaframe.data

{u'SKYSUB': {u'PARAM': {'PCHI_RESID': 0.05},
  u'QA': {'MED_RESID': 0.7486834745211439, 'NBAD_PCHI': 38, 'NSKY_FIB': 45}},
 'camera': 'b0',
 'flavor': 'science'}

### Now Edison
    [edison ~] source env.sh
    [edison ~] cd Python/desispec
    [edison ~] git pull
    [edison ~] cd
    [edison ~] module use ~xavier/modules
    [edison ~] module switch desispec/qa_exposure
    [edison ~] cd qa_test

    desi_compute_sky.py \
        --infile /project/projectdirs/desi/spectro/redux/sjb/dogwood/exposures/20150211/00000002/frame-b0-00000002.fits \
        --fibermap /project/projectdirs/desi/spectro/sim/alpha-5/20150211/fibermap-00000002.fits \
        --fiberflat /project/projectdirs/desi/spectro/redux/sjb/dogwood/calib2d/20150211/fiberflat-b0-00000001.fits \
        --outfile sky-b0-00000002.fits \
        --qafile qa-b0-00000002.yaml \
        --qafig qa-b0-00000002.pdf \        
        > qa_tst.log
